In [1]:
import logging

import numpy as np
import pandas as pd
from rich import print

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression as SklearnLogisticRegression

%load_ext rich

logging.basicConfig(level=logging.INFO)


In [2]:
data = pd.read_csv("Default.csv")

data["student"] = data["student"].map({"No": 0, "Yes": 1})
data["default"] = data["default"].map({"No": 0, "Yes": 1})

data


,default,student,balance,income
0,0,0,729.526495,44361.625074
1,0,1,817.180407,12106.134700
2,0,0,1073.549164,31767.138947
3,0,0,529.250605,35704.493935
4,0,0,785.655883,38463.495879
...,...,...,...,...
9995,0,0,711.555020,52992.378914
9996,0,0,757.962918,19660.721768
9997,0,0,845.411989,58636.156984
9998,0,0,1569.009053,36669.112365


In [3]:
X, y = data.drop("default", axis=1), data["default"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

X_train.shape, X_test.shape


((7000, 3), (3000, 3))

In [4]:
# Standardize the data for logistic regression
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Problem 1: Logistic Regression using Newton's Method

## Custom implementation

### Training sample
For a training sample, the components are defined as:

#### Sigmoid
$$
\begin{align*}
\sigma(z) &= \frac{1}{1 + e^{-z}} \\
\end{align*}
$$

#### Derivative of sigmoid

The derivative of the sigmoid function is given by:

$$
\begin{align*}
\frac{d\sigma(z)}{dz} &= \sigma(z) \cdot (1 - \sigma(z)) \\
\end{align*}
$$

#### Log-likelihood

$$
\begin{align*}
P(y|x; w ) &= \sigma_{w}(x)^{y} \cdot (1 - \sigma_{w}(x))^{1 - y} \\
L(w) &= \prod_{i=1}^{n} P(y_{i}|x_{i}; w) \\
L(w) &= \prod_{i=1}^{n} \sigma_{w}(x_{i})^{y_{i}} \cdot (1 - \sigma_{w}(x_{i}))^{1 - y_{i}} \\

\ell(w) &= \log L(w) \\
\ell(w) &= \sum_{i=1}^{n} \log \left( \sigma_{w}(x_{i})^{y_{i}} \cdot (1 - \sigma_{w}(x_{i}))^{1 - y_{i}} \right) \\
\ell(w) &= \sum_{i=1}^{n} \left( y_{i} \log \sigma_{w}(x_{i}) + (1 - y_{i}) \log (1 - \sigma_{w}(x_{i})) \right) \\
\end{align*}
$$

#### Derivative of log-likelihood

$$
\begin{align*}
\frac{\partial \ell(w)}{\partial w_i} &= \left( y \frac{1}{\sigma_{w}(x)} - (1 - y) \frac{1}{1 - \sigma_{w}(x)} \right) \frac{\partial \sigma_{w}(x)}{\partial w_i} \\
&= \left( y \frac{1}{\sigma_{w}(x)} - (1 - y) \frac{1}{1 - \sigma_{w}(x)} \right) \sigma_{w}(x) \cdot (1 - \sigma_{w}(x)) \frac{\partial w^T x}{\partial w_i} \\
&= \left( y (1 - \sigma_{w}(x)) - (1 - y) \sigma_{w}(x) \right) x_i \\
\frac{\partial \ell(w)}{\partial w_i} &= \left( y - \sigma_{w}(x) \right) x_i \\
\end{align*}
$$

#### Gradient vector

Extending the above result to the entire vector of weights $w$, we get the gradient vector as follows:

$$
\begin{align*}
\nabla \ell(w) &= \left( y - \sigma_{w}(x) \right) x \\
\end{align*}
$$

#### Second derivative of log-likelihood

$$
\begin{align*}
\frac{\partial^2 \ell(w)}{\partial w_i \partial w_j} &= \frac{\partial}{\partial w_j} \left( \frac{\partial \ell(w)}{\partial w_i} \right) \\
&= \frac{\partial}{\partial w_j} \left( \left( y - \sigma_{w}(x) \right) x_i \right) \\
&= - \frac{\partial \sigma_{w}(x)}{\partial w_j} x_i \left( y - \sigma_{w}(x) \right) \\
\frac{\partial^2 \ell(w)}{\partial w_i \partial w_j} &= - \sigma_{w}(x) \cdot (1 - \sigma_{w}(x)) \cdot x_i \cdot x_j \\
\end{align*}
$$

#### Hessian matrix

Extending the second derivative to the entire vector of weights $w$ with $d$ features, we get the Hessian matrix as follows:

$$
\begin{align*}
H(w) = \nabla^2 \ell(w) &= \begin{bmatrix}
\frac{\partial^2 \ell(w)}{\partial w_1^2} & \frac{\partial^2 \ell(w)}{\partial w_1 \partial w_2} & \cdots & \frac{\partial^2 \ell(w)}{\partial w_1 \partial w_d} \\
\frac{\partial^2 \ell(w)}{\partial w_2 \partial w_1} & \frac{\partial^2 \ell(w)}{\partial w_2^2} & \cdots & \frac{\partial^2 \ell(w)}{\partial w_2 \partial w_d} \\
\vdots & \vdots & \ddots & \vdots \\
\frac{\partial^2 \ell(w)}{\partial w_d \partial w_1} & \frac{\partial^2 \ell(w)}{\partial w_d \partial w_2} & \cdots & \frac{\partial^2 \ell(w)}{\partial w_d^2} \\
\end{bmatrix} \\

\nabla^2 \ell(w) &= - \sigma_{w}(x) \cdot (1 - \sigma_{w}(x)) \cdot x \cdot x^T \\
\end{align*}
$$

### Vectorized form
For a given matrix of features $X$ and a vector of labels $y$, the above can be written as:

#### Log-likelihood

$$
\begin{align*}
\ell(w) &= \sum_{i=1}^{n} \left( y_{i} \log \mu_{i} + (1 - y_{i}) \log (1 - \mu_{i}) \right) \\
\mu_{i} &= \sigma_{w}(x_{i}) = \frac{1}{1 + e^{-w^T x_{i}}}
\end{align*}
$$

#### Gradient vector
$$
\begin{align*}
\nabla \ell(w) &= X^T \cdot (\mu - y)
\end{align*}
$$

#### Hessian matrix
$$
\begin{align*}
H(w) &= X^T \cdot R \cdot X \\
R &= \text{diag}(\mu \cdot (1 - \mu))
\end{align*}



### Newton's method

The update rule for Newton's method is then given by:

$$
\begin{align*}
w_{t+1} = w_{t} - \gamma (\nabla^2 \ell(w_t))^{-1} \nabla \ell(w_t)
\end{align*}
$$,

where:
- $w_{t+1}$ is the updated estimate of the weights
- $w_{t}$ is the current estimate of the weights (at iteration $t$)
- $\nabla^2 \ell(w_t)^{-1}$ is the inverse of the Hessian matrix at iteration $t$
- $\nabla \ell(w_t)$ is the gradient vector at iteration $t$
- $\gamma$ is the learning rate (default value is 1)

In [5]:
class LogisticRegression:
    def __init__(self, lr=1, n_iters=1000, tol=1e-6, random_state=42, debug=False):
        self.lr = lr
        self.n_iters = n_iters
        self.weights = None
        self.tol = tol

        self.random = np.random.RandomState(random_state)

        self._logger = logging.getLogger(self.__class__.__name__)

        self._logger.setLevel(logging.DEBUG if debug else logging.INFO)

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def _log_likelihood(self, y, y_pred):
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        return -np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

    def _gradient(self, X, y, weights):
        y_pred = self._sigmoid(X @ weights)
        gradient = X.T @ (y_pred - y)

        self._logger.debug(f"Gradient shapes: {X.T.shape} @ {(y_pred - y).shape}")
        return gradient

    def _hessian(self, X, weights):
        S = np.diag(self._sigmoid(X @ weights) * (1 - self._sigmoid(X @ weights)))
        self._logger.debug(f"Hessian shapes: {X.T.shape} @ {S.shape} @ {X.shape}")

        hessian = X.T @ S @ X

        return hessian

    def _inverse(self, H):
        try:
            self._logger.debug("Calculating inverse of Hessian matrix...")
            inverse = np.linalg.inv(H)
            return inverse

        except np.linalg.LinAlgError:
            self._logger.error("Hessian matrix is not invertible! Exiting...")
            raise

    def fit(self, X, y):
        n_samples, n_features = X.shape

        self.weights = self.random.rand(n_features)

        self._logger.debug("Starting optimization using Newton's method...")

        for _ in range(self.n_iters):
            self._logger.debug(f"Iteration {_}; Weights: {self.weights}")
            current_loss = self._log_likelihood(y, self._sigmoid(X @ self.weights))

            grad = self._gradient(X, y, self.weights)
            hess = self._hessian(X, self.weights)
            inv_hess = self._inverse(hess)

            self.weights -= self.lr * inv_hess @ grad

            new_loss = self._log_likelihood(y, self._sigmoid(X @ self.weights))

            self._logger.debug(f"Loss: {current_loss} -> {new_loss}")

            if np.abs(current_loss - new_loss) < self.tol:
                self._logger.info(
                    f"Converged at iteration {_}! Loss: {new_loss}. Exiting..."
                )
                break

    def predict(self, X):
        return np.round(self._sigmoid(X @ self.weights))


In [6]:
model = LogisticRegression(n_iters=100, random_state=42, tol=1e-40, debug=True)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(f"Accuracy: {np.mean(y_test == y_pred)}")


DEBUG:LogisticRegression:Starting optimization using Newton's method...
DEBUG:LogisticRegression:Iteration 0; Weights: [0.37454012 0.95071431 0.73199394]
DEBUG:LogisticRegression:Gradient shapes: (3, 7000) @ (7000,)
DEBUG:LogisticRegression:Hessian shapes: (3, 7000) @ (7000, 7000) @ (7000, 3)
DEBUG:LogisticRegression:Calculating inverse of Hessian matrix...
DEBUG:LogisticRegression:Loss: 0.7551094257526881 -> 0.6960244936586023
DEBUG:LogisticRegression:Iteration 1; Weights: [-0.1586134   0.01140327 -0.25077335]
DEBUG:LogisticRegression:Gradient shapes: (3, 7000) @ (7000,)
DEBUG:LogisticRegression:Hessian shapes: (3, 7000) @ (7000, 7000) @ (7000, 3)
DEBUG:LogisticRegression:Calculating inverse of Hessian matrix...
DEBUG:LogisticRegression:Loss: 0.6960244936586023 -> 0.6844282282483372
DEBUG:LogisticRegression:Iteration 2; Weights: [-0.00991241  0.26823811  0.01507877]
DEBUG:LogisticRegression:Gradient shapes: (3, 7000) @ (7000,)
DEBUG:LogisticRegression:Hessian shapes: (3, 7000) @ (7000

Accuracy: 0.5303333333333333

In [7]:
model.weights

array([-0.01264089,  0.27076021,  0.01132372])

## Scikit-learn implementation

In [8]:
sklearn_model = SklearnLogisticRegression(
    fit_intercept=False, random_state=42, n_jobs=-1, verbose=3, penalty=None
)

sklearn_model.fit(X_train, y_train)

y_pred = sklearn_model.predict(X_test)

print(f"Accuracy: {np.mean(y_test == y_pred)}")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    5.6s finished


Accuracy: 0.5303333333333333

In [9]:
sklearn_model.coef_

array([[-0.01264092,  0.27076015,  0.01132377]])

In [10]:
np.allclose(model.weights, sklearn_model.coef_[0], atol=1e-3, rtol=1e-3)

True

# Problem 2: Logistic Regression using Mini-batch Stochastic Gradient Descent

In [11]:
class LogisticRegressionSGD:
    def __init__(
        self,
        lr=0.01,
        n_iters=1000,
        batch_size=8,
        tol=1e-6,
        random_state=42,
        debug=False,
    ):
        self.lr = lr
        self.n_iters = n_iters
        self.batch_size = batch_size
        self.weights = None
        self.tol = tol

        self.random = np.random.RandomState(random_state)

        self._logger = logging.getLogger(self.__class__.__name__)

        self._logger.setLevel(logging.DEBUG if debug else logging.INFO)

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def _log_likelihood(self, y, y_pred):
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        return -np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

    def _gradient(self, X, y, y_pred):
        gradient = X.T @ (y_pred - y)

        self._logger.debug(f"Gradient shapes: {X.T.shape} @ {(y_pred - y).shape}")
        return gradient

    def _get_batch(self, X, y):
        indices = self.random.choice(X.shape[0], self.batch_size, replace=False)
        return X[indices], y[indices]

    def fit(self, X, y):
        n_samples, n_features = X.shape

        self.weights = self.random.rand(n_features)

        self._logger.debug(
            f"Starting optimization using Stochastic Gradient Descent with batch size {self.batch_size}..."
        )

        loss_history = []

        for _ in range(self.n_iters):
            self._logger.debug(f"Iteration {_}; Weights: {self.weights}")

            X_batch, y_batch = self._get_batch(X, y)

            current_loss = self._log_likelihood(y_batch, self.predict(X_batch))

            y_pred = self.predict(X_batch)

            grad = self._gradient(X_batch, y_batch, y_pred)

            self.weights -= self.lr * grad

            new_loss = self._log_likelihood(y_batch, self.predict(X_batch))

            self._logger.debug(f"Batch loss: {current_loss} -> {new_loss}")

            if _ % (self.n_iters // 10) == 0:
                loss_history.append(self._log_likelihood(y, self.predict(X)))
                self._logger.debug(f'Training loss: {loss_history[-1]}')
                if (len(loss_history) > 1) and (
                    np.abs(loss_history[-1] - loss_history[-2]) < self.tol
                ):
                    self._logger.info(
                        f"Converged at iteration {_}! Final loss: {loss_history[-1]}. Exiting..."
                    )
                    break

    def predict(self, X):
        return self._sigmoid(X @ self.weights)


In [12]:
model_mini_batch_sgd = LogisticRegressionSGD(
    lr=0.05,
    n_iters=1000,
    batch_size=16,
    tol=1e-4,
    random_state=42,
    debug=True,
)

### Batch size: 16

In [13]:
model_mini_batch_sgd.fit(X_train, y_train)

y_pred = (model_mini_batch_sgd.predict(X_test) > 0.5).astype(int)

print(
    f"Accuracy (batch size {model_mini_batch_sgd.batch_size}): {np.mean(y_test == y_pred)}"
)

model_mini_batch_sgd.weights

DEBUG:LogisticRegressionSGD:Starting optimization using Stochastic Gradient Descent with batch size 16...
DEBUG:LogisticRegressionSGD:Iteration 0; Weights: [0.37454012 0.95071431 0.73199394]
DEBUG:LogisticRegressionSGD:Gradient shapes: (3, 16) @ (16,)
DEBUG:LogisticRegressionSGD:Batch loss: 0.8081901669910405 -> 0.7206984550722549
DEBUG:LogisticRegressionSGD:Iteration 1; Weights: [0.16388382 0.7688857  0.84288125]
DEBUG:LogisticRegressionSGD:Gradient shapes: (3, 16) @ (16,)
DEBUG:LogisticRegressionSGD:Batch loss: 0.7514496166036567 -> 0.7395605057237727
DEBUG:LogisticRegressionSGD:Iteration 2; Weights: [0.19158675 0.67965373 0.80765548]
DEBUG:LogisticRegressionSGD:Gradient shapes: (3, 16) @ (16,)
DEBUG:LogisticRegressionSGD:Batch loss: 0.8995690040823856 -> 0.8037186934397615
DEBUG:LogisticRegressionSGD:Iteration 3; Weights: [0.36530953 0.5400107  0.62589004]
DEBUG:LogisticRegressionSGD:Gradient shapes: (3, 16) @ (16,)
DEBUG:LogisticRegressionSGD:Batch loss: 0.7182747335913167 -> 0.701

Accuracy (batch size 16): 0.5346666666666666

array([-0.14117608,  0.25525528,  0.08341634])

### Batch size: 32

In [14]:
model_mini_batch_sgd.batch_size = 32

model_mini_batch_sgd.fit(X_train, y_train)

y_pred = (model_mini_batch_sgd.predict(X_test) > 0.5).astype(int)

print(
    f"Accuracy (batch size {model_mini_batch_sgd.batch_size}): {np.mean(y_test == y_pred)}"
)

model_mini_batch_sgd.weights

DEBUG:LogisticRegressionSGD:Starting optimization using Stochastic Gradient Descent with batch size 32...
DEBUG:LogisticRegressionSGD:Iteration 0; Weights: [0.09446087 0.87986764 0.93156588]
DEBUG:LogisticRegressionSGD:Gradient shapes: (3, 32) @ (32,)
DEBUG:LogisticRegressionSGD:Batch loss: 1.0554923416220081 -> 0.8078422855194473
DEBUG:LogisticRegressionSGD:Iteration 1; Weights: [0.32716635 0.5697657  0.3560656 ]
DEBUG:LogisticRegressionSGD:Gradient shapes: (3, 32) @ (32,)
DEBUG:LogisticRegressionSGD:Batch loss: 0.700800420730354 -> 0.6925412169778096
DEBUG:LogisticRegressionSGD:Iteration 2; Weights: [0.29770808 0.50318754 0.25835495]
DEBUG:LogisticRegressionSGD:Gradient shapes: (3, 32) @ (32,)
DEBUG:LogisticRegressionSGD:Batch loss: 0.7678752899537785 -> 0.6750564431915961
DEBUG:LogisticRegressionSGD:Iteration 3; Weights: [-0.08617918  0.21788951  0.49176266]
DEBUG:LogisticRegressionSGD:Gradient shapes: (3, 32) @ (32,)
DEBUG:LogisticRegressionSGD:Batch loss: 0.7009084413952312 -> 0.6

Accuracy (batch size 32): 0.524

array([-0.25696972,  0.36799799,  0.11610754])

### Batch size: 64

In [15]:
model_mini_batch_sgd.batch_size = 64

model_mini_batch_sgd.fit(X_train, y_train)

y_pred = (model_mini_batch_sgd.predict(X_test) > 0.5).astype(int)

print(
    f"Accuracy (batch size {model_mini_batch_sgd.batch_size}): {np.mean(y_test == y_pred)}"
)

model_mini_batch_sgd.weights

DEBUG:LogisticRegressionSGD:Starting optimization using Stochastic Gradient Descent with batch size 64...
DEBUG:LogisticRegressionSGD:Iteration 0; Weights: [0.40206487 0.61825913 0.94610843]
DEBUG:LogisticRegressionSGD:Gradient shapes: (3, 64) @ (64,)
DEBUG:LogisticRegressionSGD:Batch loss: 0.6798366155211923 -> 0.6696089125608675
DEBUG:LogisticRegressionSGD:Iteration 1; Weights: [0.29114445 0.53956508 0.80826291]
DEBUG:LogisticRegressionSGD:Gradient shapes: (3, 64) @ (64,)
DEBUG:LogisticRegressionSGD:Batch loss: 0.7299354764328279 -> 0.7027252835041318
DEBUG:LogisticRegressionSGD:Iteration 2; Weights: [0.47016683 0.47528339 0.40521193]
DEBUG:LogisticRegressionSGD:Gradient shapes: (3, 64) @ (64,)
DEBUG:LogisticRegressionSGD:Batch loss: 0.7056361292591835 -> 0.6911599708820593
DEBUG:LogisticRegressionSGD:Iteration 3; Weights: [0.26211306 0.46856923 0.29932877]
DEBUG:LogisticRegressionSGD:Gradient shapes: (3, 64) @ (64,)
DEBUG:LogisticRegressionSGD:Batch loss: 0.6835317427549462 -> 0.678

Accuracy (batch size 64): 0.552

array([0.03908224, 0.16587501, 0.14418547])